In [ ]:

import requests

repo = "PrismaX/PhysUniBench"
filename = "PhysUnivBench_en_MCQ.json"
url = f"https://huggingface.co/datasets/{repo}/raw/main/{filename}"

r = requests.get(url, timeout=30)
r.raise_for_status()
data = r.json()
print(f"{filename} -> {len(data)} questions")


PhysUnivBench_en_MCQ.json -> 393 questions


In [ ]:
import requests

repo = "PrismaX/PhysUniBench"
filename = "PhysUnivBench_en_OE.json"
url = f"https://huggingface.co/datasets/{repo}/raw/main/{filename}"

r = requests.get(url, timeout=30)
r.raise_for_status()
data = r.json()   # file is a JSON array
print(f"{filename} -> {len(data)} questions")


PhysUnivBench_en_OE.json -> 629 questions


In [ ]:

import argparse
import json
import random
import sys
from io import StringIO
from typing import Any, Dict, List, Optional

HF_RAW_TEMPLATE = "https://huggingface.co/datasets/{repo}/raw/{revision}/{filename}"
REPO = "PrismaX/PhysUniBench"
REVISION = "main"
FILENAME = "PhysUnivBench_en_MCQ.json"  # correct filename (note the 'v')

def load_text_from_local(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def load_text_from_remote(repo: str = REPO, filename: str = FILENAME, revision: str = REVISION, timeout: int = 30) -> str:
    import requests
    url = HF_RAW_TEMPLATE.format(repo=repo, filename=filename, revision=revision)
    resp = requests.get(url, timeout=timeout)
    resp.raise_for_status()
    return resp.text

def parse_json_or_jsonl(text: str) -> List[Dict[str, Any]]:
    try:
        data = json.loads(text)
        if isinstance(data, list):
            return data
        if isinstance(data, dict):
            for k in ("data", "questions", "items"):
                if k in data and isinstance(data[k], list):
                    return data[k]
            list_fields = [(k, v) for k, v in data.items() if isinstance(v, list)]
            if list_fields:
                largest = max(list_fields, key=lambda kv: len(kv[1]))[1]
                return largest
            return [data]
    except json.JSONDecodeError:
        pass

    items = []
    for line in StringIO(text):
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            if isinstance(obj, dict):
                for k in ("data","questions","items"):
                    if k in obj and isinstance(obj[k], list):
                        items.extend(obj[k])
                        break
                else:
                    items.append(obj)
            elif isinstance(obj, list):
                items.extend(obj)
        except json.JSONDecodeError:
            continue
    return items

def extract_field(item: Dict[str, Any], candidates: List[str]) -> Optional[Any]:
    for c in candidates:
        if c in item:
            return item[c]
    return None

def get_question_text(item: Dict[str, Any]) -> str:
    txt = extract_field(item, ["question", "question_text", "stem", "content", "prompt", "body", "q"])
    if isinstance(txt, dict):
        return txt.get("text") or txt.get("value") or json.dumps(txt, ensure_ascii=False)
    if txt is None:
        title = item.get("title") or item.get("name")
        body = item.get("body") or item.get("description")
        if title and body:
            return f"{title}\n{body}"
        return json.dumps(item, ensure_ascii=False)[:200] + "..."
    return str(txt)

def get_choices(item: Dict[str, Any]) -> List[str]:
    choices = extract_field(item, ["choices", "options", "alternatives", "answers", "options_list"])
    if choices is None:
        letters = [k for k in sorted(item.keys()) if k.upper() in ("A","B","C","D","E","F")]
        if letters:
            return [item[k] for k in letters]
        return []
    if isinstance(choices, dict):
        sorted_keys = sorted(choices.keys(), key=lambda k: k)
        return [choices[k] if choices[k] is not None else "" for k in sorted_keys]
    if isinstance(choices, list):
        out = []
        for ch in choices:
            if isinstance(ch, str):
                out.append(ch)
            elif isinstance(ch, dict):
                out.append(ch.get("text") or ch.get("choice") or ch.get("label") or json.dumps(ch, ensure_ascii=False))
            else:
                out.append(str(ch))
        return out
    return [str(choices)]

def find_correct_index(item: Dict[str, Any], choices: List[str]) -> Optional[int]:
    ans = extract_field(item, ["answer", "answer_key", "correct_answer", "correct", "label", "answerKey", "ans"])
    if ans is None:
        return None
    if isinstance(ans, int):
        if 0 <= ans < len(choices):
            return ans
        if 1 <= ans <= len(choices):
            return ans - 1
        return None
    if isinstance(ans, str):
        s = ans.strip()
        if len(s) == 1 and s.upper() >= "A" and ord(s.upper()) - ord("A") < len(choices):
            return ord(s.upper()) - ord("A")
        if s.isdigit():
            idx = int(s)
            if 0 <= idx < len(choices):
                return idx
            if 1 <= idx <= len(choices):
                return idx - 1
        for i, ch in enumerate(choices):
            if ch and s.lower() == str(ch).strip().lower():
                return i
    if isinstance(ans, dict):
        if "label" in ans:
            return find_correct_index({"_tmp": ans["label"]}, choices)
        if "index" in ans and isinstance(ans["index"], int):
            return ans["index"]
    return None

def print_mcq(item: Dict[str, Any], idx: int, reveal_answer: bool = False) -> None:
    qtext = get_question_text(item)
    choices = get_choices(item)
    correct_idx = find_correct_index(item, choices)

    print(f"Q{idx}. {qtext}")
    if not choices:
        print("   (no choices found)\n")
        return

    for i, ch in enumerate(choices):
        letter = chr(ord("A") + i)
        print(f"   {letter}. {ch}")

    if reveal_answer:
        if correct_idx is not None and 0 <= correct_idx < len(choices):
            print(f"   >> Correct: {chr(ord('A')+correct_idx)}. {choices[correct_idx]}")
        else:
            raw_ans = extract_field(item, ["answer", "answer_key", "correct_answer", "correct", "label","answerKey","ans"])
            if raw_ans is not None:
                print(f"   >> Correct (raw): {raw_ans}")
            else:
                print("   >> Correct: (unknown)")
    print()

def print_some_mcqs(local_path: Optional[str] = None, n: int = 5, first: bool = False, seed: Optional[int] = None, reveal: bool = False):
    if local_path:
        text = load_text_from_local(local_path)
    else:
        text = load_text_from_remote()
    items = parse_json_or_jsonl(text)
    if not items:
        print("No items parsed from file.")
        return
    total = len(items)
    print(f"Loaded {total} items. Printing {min(n, total)} MCQs.\n")
    indices = list(range(total))
    if first:
        chosen = indices[:n]
    else:
        if seed is not None:
            random.seed(seed)
        chosen = random.sample(indices, min(n, total))
    for i, idx in enumerate(chosen, start=1):
        print_mcq(items[idx], i, reveal_answer=reveal)

def main(argv=None):
    ap = argparse.ArgumentParser()
    ap.add_argument("--local", "-l", help="Path to local JSON / JSONL file (if omitted, fetches from HuggingFace raw URL)")
    ap.add_argument("--n", "-n", type=int, default=5, help="Number of MCQs to print")
    ap.add_argument("--first", action="store_true", help="Print the first N items instead of random sample")
    ap.add_argument("--seed", type=int, default=None, help="Random seed for sampling")
    ap.add_argument("--reveal", action="store_true", help="Reveal correct answers")

    if argv is None:
        args, unknown = ap.parse_known_args()
    else:
        args = ap.parse_args(argv)

    print_some_mcqs(local_path=args.local, n=args.n, first=args.first, seed=args.seed, reveal=args.reveal)

if __name__ == "__main__":
    main()


Loaded 393 items. Printing 5 MCQs.

Q1. An air - filled toroidal solenoid has a mean radius $r = 14.5\ cm=0.145\ m$ and a cross - sectional area $A = 5.05\ cm^{2}=5.05\times10^{- 4}\ m^{2}$. The current flowing through it is $I = 11.7\ A$, and it is desired that the energy stored within the solenoid be at least $U = 0.385\ J$.
### Part A
What is the least number of turns that the winding must have? Express your answer numerically, as a whole number, to three significant figures.
View Available Hint(s)
Submit
turns
<image>
   A. A. N = 2353

B. N = 159

C. N = 2830

D. N = 130.32

Q2. Calculate the magnitude and direction of the Coulomb force on each of the three charges shown in the figure below.
1.50μC charge magnitude
1.50μC charge direction
-2.00μC charge magnitude
-2.00μC charge direction
6.00 μC charge magnitude  N
6.00 μC charge direction  ---Select---
1.50 μC charge magnitude  N
1.50 μC charge direction  ---Select---
-2.00 μC charge magnitude  N
-2.00 μC charge direction  ---Sel

In [ ]:

import argparse
import json
import random
import sys
from io import StringIO
from typing import Any, Dict, List, Optional

HF_RAW_TEMPLATE = "https://huggingface.co/datasets/{repo}/raw/{revision}/{filename}"
REPO = "PrismaX/PhysUniBench"
REVISION = "main"
FILENAME = "PhysUnivBench_en_OE.json"  # correct filename (note the 'v')

def load_text_from_local(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def load_text_from_remote(repo: str = REPO, filename: str = FILENAME, revision: str = REVISION, timeout: int = 30) -> str:
    import requests
    url = HF_RAW_TEMPLATE.format(repo=repo, filename=filename, revision=revision)
    resp = requests.get(url, timeout=timeout)
    resp.raise_for_status()
    return resp.text

def parse_json_or_jsonl(text: str) -> List[Dict[str, Any]]:
    try:
        data = json.loads(text)
        if isinstance(data, list):
            return data
        if isinstance(data, dict):
            for k in ("data", "questions", "items"):
                if k in data and isinstance(data[k], list):
                    return data[k]
            list_fields = [(k, v) for k, v in data.items() if isinstance(v, list)]
            if list_fields:
                largest = max(list_fields, key=lambda kv: len(kv[1]))[1]
                return largest
            return [data]
    except json.JSONDecodeError:
        pass

    items = []
    for line in StringIO(text):
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            if isinstance(obj, dict):
                for k in ("data","questions","items"):
                    if k in obj and isinstance(obj[k], list):
                        items.extend(obj[k])
                        break
                else:
                    items.append(obj)
            elif isinstance(obj, list):
                items.extend(obj)
        except json.JSONDecodeError:
            continue
    return items

def extract_field(item: Dict[str, Any], candidates: List[str]) -> Optional[Any]:
    for c in candidates:
        if c in item:
            return item[c]
    return None

def get_question_text(item: Dict[str, Any]) -> str:
    txt = extract_field(item, ["question", "question_text", "stem", "content", "prompt", "body", "q"])
    if isinstance(txt, dict):
        return txt.get("text") or txt.get("value") or json.dumps(txt, ensure_ascii=False)
    if txt is None:
        title = item.get("title") or item.get("name")
        body = item.get("body") or item.get("description")
        if title and body:
            return f"{title}\n{body}"
        return json.dumps(item, ensure_ascii=False)[:200] + "..."
    return str(txt)

def get_choices(item: Dict[str, Any]) -> List[str]:
    choices = extract_field(item, ["choices", "options", "alternatives", "answers", "options_list"])
    if choices is None:
        letters = [k for k in sorted(item.keys()) if k.upper() in ("A","B","C","D","E","F")]
        if letters:
            return [item[k] for k in letters]
        return []
    if isinstance(choices, dict):
        sorted_keys = sorted(choices.keys(), key=lambda k: k)
        return [choices[k] if choices[k] is not None else "" for k in sorted_keys]
    if isinstance(choices, list):
        out = []
        for ch in choices:
            if isinstance(ch, str):
                out.append(ch)
            elif isinstance(ch, dict):
                out.append(ch.get("text") or ch.get("choice") or ch.get("label") or json.dumps(ch, ensure_ascii=False))
            else:
                out.append(str(ch))
        return out
    return [str(choices)]

def find_correct_index(item: Dict[str, Any], choices: List[str]) -> Optional[int]:
    ans = extract_field(item, ["answer", "answer_key", "correct_answer", "correct", "label", "answerKey", "ans"])
    if ans is None:
        return None
    if isinstance(ans, int):
        if 0 <= ans < len(choices):
            return ans
        if 1 <= ans <= len(choices):
            return ans - 1
        return None
    if isinstance(ans, str):
        s = ans.strip()
        if len(s) == 1 and s.upper() >= "A" and ord(s.upper()) - ord("A") < len(choices):
            return ord(s.upper()) - ord("A")
        if s.isdigit():
            idx = int(s)
            if 0 <= idx < len(choices):
                return idx
            if 1 <= idx <= len(choices):
                return idx - 1
        for i, ch in enumerate(choices):
            if ch and s.lower() == str(ch).strip().lower():
                return i
    if isinstance(ans, dict):
        if "label" in ans:
            return find_correct_index({"_tmp": ans["label"]}, choices)
        if "index" in ans and isinstance(ans["index"], int):
            return ans["index"]
    return None

def print_mcq(item: Dict[str, Any], idx: int, reveal_answer: bool = False) -> None:
    qtext = get_question_text(item)
    choices = get_choices(item)
    correct_idx = find_correct_index(item, choices)

    print(f"Q{idx}. {qtext}")
    if not choices:
        print("   (no choices found)\n")
        return

    for i, ch in enumerate(choices):
        letter = chr(ord("A") + i)
        print(f"   {letter}. {ch}")

    if reveal_answer:
        if correct_idx is not None and 0 <= correct_idx < len(choices):
            print(f"   >> Correct: {chr(ord('A')+correct_idx)}. {choices[correct_idx]}")
        else:
            raw_ans = extract_field(item, ["answer", "answer_key", "correct_answer", "correct", "label","answerKey","ans"])
            if raw_ans is not None:
                print(f"   >> Correct (raw): {raw_ans}")
            else:
                print("   >> Correct: (unknown)")
    print()

def print_some_mcqs(local_path: Optional[str] = None, n: int = 5, first: bool = False, seed: Optional[int] = None, reveal: bool = False):
    if local_path:
        text = load_text_from_local(local_path)
    else:
        text = load_text_from_remote()
    items = parse_json_or_jsonl(text)
    if not items:
        print("No items parsed from file.")
        return
    total = len(items)
    print(f"Loaded {total} items. Printing {min(n, total)} MCQs.\n")
    indices = list(range(total))
    if first:
        chosen = indices[:n]
    else:
        if seed is not None:
            random.seed(seed)
        chosen = random.sample(indices, min(n, total))
    for i, idx in enumerate(chosen, start=1):
        print_mcq(items[idx], i, reveal_answer=reveal)

def main(argv=None):
    ap = argparse.ArgumentParser()
    ap.add_argument("--local", "-l", help="Path to local JSON / JSONL file (if omitted, fetches from HuggingFace raw URL)")
    ap.add_argument("--n", "-n", type=int, default=5, help="Number of MCQs to print")
    ap.add_argument("--first", action="store_true", help="Print the first N items instead of random sample")
    ap.add_argument("--seed", type=int, default=None, help="Random seed for sampling")
    ap.add_argument("--reveal", action="store_true", help="Reveal correct answers")

    # parse_known_args allows Jupyter/Colab injected args (like -f) to be ignored
    if argv is None:
        args, unknown = ap.parse_known_args()
    else:
        args = ap.parse_args(argv)

    print_some_mcqs(local_path=args.local, n=args.n, first=args.first, seed=args.seed, reveal=args.reveal)

if __name__ == "__main__":
    main()


Loaded 629 items. Printing 5 MCQs.

Q1. how to determine the x - component and y - component of vector $\vec{A}$, denoted $A_x$ and $A_y$.
What is the magnitude of the component vector $\vec{A}_x$ shown in (Figure 1)? Express your answer in meters to one significant figure.
Previous Answers: Incorrect; Try Again
Part B
What is the sign of the y - component $A_y$ of vector $\vec{A}$ shown in (Figure 1)?
positive
negative
Previous Answers: Correct
<image>
   (no choices found)

Q2. VP 21.4.1
Charge $q_1 = + 5.60nC$ is on the $x$-axis at $x = 0$ and charge $q_2=-2.00nC$ is on the $x$-axis at $x = 6.00cm$. Find the $x$-component of total electric force exerted by $q_1$ and $q_2$ on a third charge $q_3=-1.50nC$ on the $x$-axis at $x = 4.00cm$.
Express your answer in newtons.
$F_x=$
Incorrect; Try Again; 5 attempts remaining
Charge $q_1 = +5.60\ nC$ is on the $x$-axis at $x = 0$ and charge $q_2=-2.00\ nC$ is on the $x$-axis at $x = 6.00\ cm$. Find the $x$-component of total electric force ex

In [ ]:


import requests
import json
import re
import io
import os
import zipfile
from PIL import Image

try:
    from tqdm import tqdm
except Exception:
    tqdm = lambda x, **k: x

# ==== CONFIG ====
RAW_JSON_URL = "https://huggingface.co/datasets/PrismaX/PhysUniBench/raw/main/PhysUnivBench_en_MCQ.json"
IMAGE_BASE_URL = "https://huggingface.co/datasets/PrismaX/PhysUniBench/raw/main/images/"

OUTPUT_JSON = "phys_unibench_mcqs.json"
OUTPUT_IMAGES_ZIP = "phys_unibench_images.zip"
START_ID = 174
# =================

def safe_get_json(url):
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()

def parse_options(options_text):
    """
    Best-effort parse of options string into dict {'a':..., 'b':..., 'c':..., 'd':...}.
    Works for forms like "A. ... B. ... C. ... D. ..." or newline-separated variants.
    """
    if not options_text:
        return {"a":"", "b":"", "c":"", "d":""}
    s = options_text.strip()
    # unify different unicode dots/spaces (best-effort)
    # find positions of A., B., C., D. (capital letters expected)
    labels = ['A','B','C','D']
    positions = []
    for lab in labels:
        m = re.search(r'(?<!\w)'+re.escape(lab)+r'\.', s)
        if m:
            positions.append((lab, m.start()))
    # If none found, try lowercase
    if not positions:
        for lab in labels:
            m = re.search(r'(?<!\w)'+lab.lower()+r'\.', s)
            if m:
                positions.append((lab, m.start()))
    # If still none, fallback: split by newlines and pick first 4 lines
    if not positions:
        parts = [line.strip() for line in s.splitlines() if line.strip()]
        out = {}
        for i, lab in enumerate(['a','b','c','d']):
            out[lab] = parts[i] if i < len(parts) else ""
        return out

    # sort by position
    positions.sort(key=lambda x: x[1])
    out = {}
    for idx, (lab, pos) in enumerate(positions):
        start = pos + 2  # skip e.g. "A."
        end = positions[idx+1][1] if idx+1 < len(positions) else len(s)
        piece = s[start:end].strip()
        # strip leading punctuation/colons/hyphens
        piece = re.sub(r'^[\s:–—\-)]*', '', piece)
        out[lab.lower()] = piece

    # ensure keys exist
    for k in ['a','b','c','d']:
        out.setdefault(k, "")

    return out

def download_and_convert_image(image_filename, target_id):
    """
    Fetch image from IMAGE_BASE_URL + image_filename (e.g. '1.jpg'),
    convert to PNG and return PNG bytes. Returns None on failure.
    """
    if not image_filename:
        return None
    url = IMAGE_BASE_URL + image_filename
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code != 200:
            # try jpg->png name variants (best-effort)
            alt = os.path.splitext(image_filename)[0] + ".png"
            resp = requests.get(IMAGE_BASE_URL + alt, timeout=60)
            if resp.status_code != 200:
                return None
        # load into PIL and convert to PNG bytes
        img = Image.open(io.BytesIO(resp.content)).convert("RGB")
        buf = io.BytesIO()
        img.save(buf, format="PNG")
        return buf.getvalue()
    except Exception as e:
        # print or log as needed
        # print(f"Image download/convert failed for {url}: {e}")
        return None

def main():
    print("Downloading JSON from dataset...")
    data = safe_get_json(RAW_JSON_URL)
    print(f"Loaded {len(data)} entries from the MCQ file.")

    output_list = []
    # Prepare zip file for images
    zf = zipfile.ZipFile(OUTPUT_IMAGES_ZIP, mode="w", compression=zipfile.ZIP_DEFLATED)
    current_id = START_ID
    for row in tqdm(data):
        qid = current_id
        current_id+=1
        # Use int/string id as-is (keep same as dataset)
        if qid is None:
            # skip if no id
            continue

        # question text
        text = row.get("question", "").strip()

        # parse options from the dataset's 'options' field (often a printable string)
        options_parsed = parse_options(row.get("options", "") or "")

        # map correct answer (dataset uses 'answer' like "A")
        correct = row.get("answer", "")
        if isinstance(correct, str):
            correct = correct.strip()
        else:
            correct = str(correct)

        # image file in dataset (e.g. "1.jpg" or "0.jpg")
        src_image_name = row.get("image", "").strip() if row.get("image") else ""

        # target image path inside JSON output (and inside zip)
        image_field_path = f"images/image{qid}.png"

        # attempt to download and convert to PNG bytes
        png_bytes = None
        if src_image_name:
            png_bytes = download_and_convert_image(src_image_name, qid)
        if png_bytes:
            # write into zip under images/image{qid}.png
            zf.writestr(image_field_path, png_bytes)
        else:
            # If missing, we still include the JSON path but no image is placed in zip.
            # Option: you can choose to set image_field_path="" instead.
            print(f"Warning: image for id={qid} ({src_image_name}) not downloaded.")

        # build output object
        out_obj = {
            "id": qid,
            "image": image_field_path,
            "text": text,
            "options": {
                "a": options_parsed.get("a", ""),
                "b": options_parsed.get("b", ""),
                "c": options_parsed.get("c", ""),
                "d": options_parsed.get("d", "")
            },
            "type": "MCQs with One Correct Answer",
            "correct_answer": correct
        }
        output_list.append(out_obj)

    zf.close()

    # save output JSON
    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(output_list, f, ensure_ascii=False, indent=2)

    print(f"Done. {len(output_list)} MCQs written to {OUTPUT_JSON}")
    print(f"Images written (when available) into {OUTPUT_IMAGES_ZIP}")

if __name__ == "__main__":
    main()


Loaded 393 entries from the MCQ file.


  1%|          | 2/393 [00:00<00:31, 12.27it/s]

  1%|          | 4/393 [00:00<00:31, 12.31it/s]

  2%|▏         | 6/393 [00:00<00:32, 11.96it/s]

  2%|▏         | 8/393 [00:00<00:31, 12.18it/s]

...

 98%|█████████▊| 387/393 [00:36<00:00, 12.51it/s]

 99%|█████████▉| 389/393 [00:36<00:00, 11.52it/s]

 99%|█████████▉| 391/393 [00:36<00:00, 11.74it/s]

100%|██████████| 393/393 [00:36<00:00, 10.70it/s]

Done. 393 MCQs written to phys_unibench_mcqs.json
Images written (when available) into phys_unibench_images.zip


In [ ]:
!pip install --quiet requests pillow tqdm


In [ ]:
import os, io, sys, json, zipfile
from typing import Optional, List
import requests
from PIL import Image

# try to import tqdm, fallback to identity
try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **kw):
        return x

# ---- CONFIG ----
HF_JSON_URL = "https://huggingface.co/datasets/PrismaX/PhysUniBench/raw/main/PhysUnivBench_en_MCQ.json"
IMAGE_BASE_URL = "https://huggingface.co/datasets/PrismaX/PhysUniBench/resolve/main/images/"
DEFAULT_START_ID = 174
DEFAULT_OUTPUT_ZIP = "phys_unibench_images.zip"
REQUEST_TIMEOUT = 30
# ----------------

def load_json(path_or_url: Optional[str]) -> List[dict]:
    """Load JSON either from a local file path or (if None) from HF_JSON_URL."""
    if path_or_url:
        if os.path.exists(path_or_url):
            with open(path_or_url, "r", encoding="utf-8") as f:
                return json.load(f)
        else:
            r = requests.get(path_or_url, timeout=REQUEST_TIMEOUT)
            r.raise_for_status()
            return r.json()
    else:
        r = requests.get(HF_JSON_URL, timeout=REQUEST_TIMEOUT)
        r.raise_for_status()
        return r.json()

def candidate_filenames(original: str) -> List[str]:
    """Return ordered candidate filenames to try (handles 0-index vs 1-index and .jpg/.png)."""
    if not original:
        return []
    base, ext = os.path.splitext(original)
    candidates = [original, base + ".png", base + ".jpg"]
    if base.isdigit():
        try:
            n = int(base)
            candidates.append(str(n + 1) + ".png")
            candidates.append(str(n + 1) + ".jpg")
        except Exception:
            pass
    # dedupe preserving order
    seen = set(); out=[]
    for c in candidates:
        if c and c not in seen:
            seen.add(c); out.append(c)
    return out

def download_and_convert(session: requests.Session, fname: str) -> Optional[bytes]:
    """Download IMAGE_BASE_URL + fname and convert to PNG bytes, or return None on failure."""
    url = IMAGE_BASE_URL + fname
    try:
        resp = session.get(url, timeout=REQUEST_TIMEOUT)
        if resp.status_code != 200 or not resp.content:
            return None
        try:
            img = Image.open(io.BytesIO(resp.content)).convert("RGB")
            buf = io.BytesIO(); img.save(buf, format="PNG")
            return buf.getvalue()
        except Exception:
            # fallback: return raw bytes (rare)
            return resp.content
    except Exception:
        return None

def extract_images_to_zip(json_path: Optional[str]=None, start_id: int=DEFAULT_START_ID,
                          output_zip: str=DEFAULT_OUTPUT_ZIP, debug: bool=False):
    """Main function to extract images into zip as images/image{ID}.png"""
    try:
        data = load_json(json_path)
    except Exception as e:
        print("Failed to load JSON:", e)
        return

    if not isinstance(data, list):
        print("JSON root is not a list; aborting.")
        return

    session = requests.Session()
    session.headers.update({"User-Agent":"physunibench-image-extractor/1.0"})

    total = len(data)
    print(f"Loaded {total} entries. Starting ID = {start_id}. Output zip = {output_zip}")

    success = 0
    failed_entries = []

    # ALWAYS create the zip (context manager ensures close)
    with zipfile.ZipFile(output_zip, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
        # a small README so the zip is never empty
        readme_text = ("PhysUniBench images ZIP\n"
                       "Images stored as images/image{ID}.png\n")
        zf.writestr("README.txt", readme_text)

        current_id = start_id
        for idx, row in enumerate(tqdm(data, desc="Downloading images")):
            qid = current_id
            current_id += 1

            # typical image key names in the dataset
            src_image_name = ""
            if isinstance(row, dict):
                src_image_name = (row.get("image") or row.get("img") or row.get("filename") or "")
            else:
                src_image_name = ""

            image_zip_path = f"images/image{qid}.png"

            if not src_image_name:
                failed_entries.append((qid, src_image_name, ["(no filename in JSON)"]))
                if debug:
                    print(f"[{qid}] no image filename in JSON for entry index {idx}")
                continue

            png_bytes = None
            tried = []
            for cand in candidate_filenames(src_image_name):
                tried.append(IMAGE_BASE_URL + cand)
                png_bytes = download_and_convert(session, cand)
                if png_bytes:
                    break

            if png_bytes:
                zf.writestr(image_zip_path, png_bytes)
                success += 1
                if debug:
                    print(f"[{qid}] downloaded -> {image_zip_path} (source: {cand})")
            else:
                failed_entries.append((qid, src_image_name, tried))
                if debug:
                    print(f"[{qid}] FAILED. Tried URLs:")
                    for u in tried:
                        print("   ", u)

    abs_path = os.path.abspath(output_zip)
    print("\nFinished.")
    print(f"ZIP file written to: {abs_path}")
    print(f"Images downloaded: {success} / {total}")
    if failed_entries:
        print(f"Failed images: {len(failed_entries)}. Use debug=True to inspect tried URLs.")
    else:
        print("All images downloaded successfully (or there were no image fields).")

# ---- RUN it here with defaults ----
# change parameters below if you want (e.g., json_path='/content/PhysUnivBench_en_MCQ.json')
extract_images_to_zip(json_path=None, start_id=174, output_zip="phys_unibench_images.zip", debug=True)


Loaded 393 entries. Starting ID = 174. Output zip = phys_unibench_images.zip


[174] downloaded -> images/image174.png (source: 0.jpg)


[175] downloaded -> images/image175.png (source: 1.jpg)


[176] downloaded -> images/image176.png (source: 2.jpg)


[177] downloaded -> images/image177.png (source: 3.jpg)


...

[562] downloaded -> images/image562.png (source: 388.jpg)


[563] downloaded -> images/image563.png (source: 389.jpg)


[564] downloaded -> images/image564.png (source: 390.jpg)


[565] downloaded -> images/image565.png (source: 391.jpg)


[566] downloaded -> images/image566.png (source: 392.jpg)

Finished.
ZIP file written to: /content/phys_unibench_images.zip
Images downloaded: 393 / 393
All images downloaded successfully (or there were no image fields).


In [ ]:

import json
from pathlib import Path

# --- START CONFIG ---
start_id = 567            # provided starting id
input_filename = "input.json"   # default input file name (you can change)
output_filename = "output_phy.json"
# --- END CONFIG ---

# Try to find the input file in the working directory; if not found, prompt Colab upload
input_path = Path(input_filename)
if not input_path.exists():
    try:
        from google.colab import files
        print(f"'{input_filename}' not found. Please upload your JSON file now (choose the original file).")
        uploaded = files.upload()  # user will upload a file
        # pick the first uploaded filename
        input_path = Path(next(iter(uploaded)))
        print(f"Uploaded: {input_path}")
    except Exception as e:
        raise FileNotFoundError(f"Could not locate '{input_filename}' and upload failed: {e}")

# Load JSON (supports a top-level list or a top-level dict)
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Normalize to a list of objects
if isinstance(data, list):
    items = data
elif isinstance(data, dict):
    # try some common wrapper keys, else treat the dict as a single item
    possible_lists = ["items", "data", "questions", "rows"]
    found = False
    for key in possible_lists:
        if key in data and isinstance(data[key], list):
            items = data[key]
            found = True
            break
    if not found:
        # treat as single object (wrap in list)
        items = [data]
else:
    raise ValueError("Unsupported JSON structure: top-level must be list or dict")

# Build output: only keep objects with subject == 'phy' (case-insensitive)
out = []
cur_id = int(start_id)
for obj in items:
    subj = str(obj.get("subject", "")).strip().lower()
    if subj == "phy":
        new_obj = {
            "id": cur_id,
            "image": None,
            "text": obj.get("question"),
            "type": obj.get("type"),
            "correct_answer": obj.get("gold")
        }
        out.append(new_obj)
        cur_id += 1

# Save output JSON (pretty-printed, ensure_ascii=False to preserve unicode)
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

print(f"Saved {len(out)} filtered items to '{output_filename}' (ids {start_id}..{cur_id-1})")

# In Colab, trigger download of the result file
try:
    from google.colab import files
    files.download(output_filename)
except Exception:
    pass


'input.json' not found. Please upload your JSON file now (choose the original file).


Saving dataset.json to dataset (2).json
Uploaded: dataset (2).json
Saved 123 filtered items to 'output_phy.json' (ids 567..689)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

import json
import os
import sys
import subprocess
import traceback

def ensure_package(pkg):
    try:
        __import__(pkg)
    except Exception:
        print(f"Package '{pkg}' not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        print(f"Installed '{pkg}'.")

def try_load_dataset():
    from datasets import load_dataset
    attempts = []
    # Try multiple common approaches to load the split
    try:
        print("Attempt: load_dataset('Cloudriver/PhyX', split='test_mini')")
        ds = load_dataset("Cloudriver/PhyX", split="test_mini")
        return ds
    except Exception as e:
        attempts.append(("split=test_mini", e))
    try:
        print("Attempt: load_dataset('Cloudriver/PhyX', 'default', split='test_mini')")
        ds = load_dataset("Cloudriver/PhyX", "default", split="test_mini")
        return ds
    except Exception as e:
        attempts.append(("config default + split", e))
    try:
        print("Attempt: load_dataset('Cloudriver/PhyX') and select ['test_mini']")
        ds_all = load_dataset("Cloudriver/PhyX")
        if "test_mini" in ds_all:
            return ds_all["test_mini"]
        else:
            raise RuntimeError(f"'test_mini' not found in dataset splits: {list(ds_all.keys())}")
    except Exception as e:
        attempts.append(("load full dataset", e))

    # If all attempts failed, raise a combined error with traces
    msg_lines = ["Failed to load dataset. Attempts:"]
    for name, err in attempts:
        msg_lines.append(f"--- {name} error: {repr(err)}")
    raise RuntimeError("\n".join(msg_lines))

def build_json_records(dataset, start_id=690):
    out = []
    cur = start_id
    # convert to list (some datasets are iterable and len() won't work)
    try:
        data_list = list(dataset)
    except Exception:
        # fallback: iterate
        data_list = []
        for i, row in enumerate(dataset):
            data_list.append(row)
    for item in data_list:
        # item may be a dict-like; use .get if available
        if hasattr(item, "get"):
            q = item.get("question") or ""
            qd = item.get("question_description") or ""
            options = item.get("options") or []
            answer = item.get("answer", None)
        else:
            # attempt attribute access
            q = getattr(item, "question", "") or ""
            qd = getattr(item, "question_description", "") or ""
            options = getattr(item, "options", []) or []
            answer = getattr(item, "answer", None)

        if q and qd:
            text = q.strip() + "\n\n" + qd.strip()
        else:
            text = (q + qd).strip()

        # normalize answer if it's list/tuple
        if isinstance(answer, (list, tuple)):
            answer_val = answer[0] if answer else None
        else:
            answer_val = answer

        record = {
            "id": cur,
            "image": f"images/image{cur}.png",
            "text": text,
            "options": options,
            "type": "MCQs with One Correct Answer",
            "correct_answer": answer_val,
        }
        out.append(record)
        cur += 1
    return out

def main(output_path="phyx_test_mini_custom.json", start_id=690):
    try:
        ensure_package("datasets")
        ensure_package("huggingface-hub")
        # import after ensure
        from datasets import __version__ as dver
        print(f"datasets version: {dver}")
    except Exception as e:
        print("Failed to ensure datasets/huggingface-hub packages:", e)
        traceback.print_exc()
        return

    try:
        print("Loading dataset Cloudriver/PhyX (test_mini)...")
        ds = try_load_dataset()
    except Exception as e:
        print("ERROR while loading dataset:")
        traceback.print_exc()
        # write an empty file and return to indicate failure but produce output
        abs_path = os.path.abspath(output_path)
        print(f"Writing empty JSON [] to {abs_path} to avoid silent failure.")
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump([], f, ensure_ascii=False, indent=2)
        return

    # convert to list and print some metadata
    try:
        ds_list = list(ds)
        n = len(ds_list)
        print(f"Successfully loaded dataset split. Number of examples: {n}")
    except Exception as e:
        print("Could not convert dataset to list; attempting to iterate.")
        traceback.print_exc()
        ds_list = []
        for i, r in enumerate(ds):
            ds_list.append(r)
        n = len(ds_list)
        print(f"Number of examples after iteration: {n}")

    print("Building JSON records...")
    records = build_json_records(ds_list, start_id=start_id)

    # write file
    abs_path = os.path.abspath(output_path)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)

    print(f"Wrote {len(records)} records to {abs_path}")
    if records:
        print("Preview of first record:")
        print(json.dumps(records[0], ensure_ascii=False, indent=2))
    else:
        print("No records produced. Check dataset loading above.")

if __name__ == "__main__":
    main()


Package 'huggingface-hub' not found. Installing...
Installed 'huggingface-hub'.
datasets version: 4.0.0
Loading dataset Cloudriver/PhyX (test_mini)...
Attempt: load_dataset('Cloudriver/PhyX', split='test_mini')
Successfully loaded dataset split. Number of examples: 1000
Building JSON records...
Wrote 1000 records to /content/phyx_test_mini_custom.json
Preview of first record:
{
  "id": 690,
  "image": "images/image690.png",
  "text": "Determine the angles\\( \\theta' \\).\n\nFigure shows a refracted light beam in linseed oil making an angle of \\( \\phi = 20.0^{\\circ} \\) with the normal line \\( NN' \\). The index of refraction of linseed oil is 1.48.",
  "options": [
    "A: \\( 28.5^{\\circ} \\)",
    "B: \\(  30.4^{\\circ} \\)",
    "C: \\( 22.3^{\\circ} \\)",
    "D: \\( 31.1^{\\circ} \\)"
  ],
  "type": "MCQs with One Correct Answer",
  "correct_answer": "C"
}


In [ ]:


import io
import json
import os
import sys
import subprocess
import zipfile
import traceback

# ---------- helper to ensure packages ----------
def ensure_package(pkg_name, import_name=None):
    import_name = import_name or pkg_name
    try:
        __import__(import_name)
    except Exception:
        print(f"Package '{pkg_name}' not found — installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        print(f"Installed '{pkg_name}'.")

# ensure required packages
for pkg in ("datasets", "Pillow", "requests", "tqdm"):
    ensure_package(pkg)

from datasets import load_dataset
from PIL import Image
import requests
from tqdm import tqdm

# ---------- configuration ----------
START_ID = 690
OUTPUT_ZIP = "phyx_images_and_data.zip"
OUTPUT_JSON_NAME = "phyx_test_mini_custom.json"
IMAGES_DIR_IN_ZIP = "images"  # images stored under images/image{ID}.png
PLACEHOLDER_SIZE = (1, 1)  # 1x1 PNG placeholder for missing images
REQUESTS_TIMEOUT = 15  # seconds

# ---------- utility functions ----------
def load_phyx_test_mini():
    """Try multiple ways to load the test_mini split."""
    attempts = []
    try:
        print("Attempting: load_dataset('Cloudriver/PhyX', split='test_mini') ...")
        ds = load_dataset("Cloudriver/PhyX", split="test_mini")
        return ds
    except Exception as e:
        attempts.append(("split=test_mini", e))
    try:
        print("Attempting: load_dataset('Cloudriver/PhyX', 'default', split='test_mini') ...")
        ds = load_dataset("Cloudriver/PhyX", "default", split="test_mini")
        return ds
    except Exception as e:
        attempts.append(("config default + split", e))
    try:
        print("Attempting: load_dataset('Cloudriver/PhyX') then select ['test_mini'] ...")
        ds_all = load_dataset("Cloudriver/PhyX")
        if "test_mini" in ds_all:
            return ds_all["test_mini"]
        else:
            raise RuntimeError(f"'test_mini' not found in splits: {list(ds_all.keys())}")
    except Exception as e:
        attempts.append(("load full dataset", e))
    # raise combined error
    msg = "Failed to load dataset. Attempts:\n"
    for name, err in attempts:
        msg += f" - {name}: {repr(err)}\n"
    raise RuntimeError(msg)

def build_text(item):
    q = item.get("question") or ""
    qd = item.get("question_description") or ""
    if q and qd:
        return q.strip() + "\n\n" + qd.strip()
    return (q + qd).strip()

def normalize_answer(ans):
    if isinstance(ans, (list, tuple)):
        return ans[0] if ans else None
    return ans

def find_image_source(item):
    """
    Try to find an image source inside the dataset example.
    Returns a tuple (type, value) where type in {'url','local_path','pil','bytes','none'}.
    """
    # direct 'image' field
    image_field = item.get("image")
    if image_field is not None:
        # If it's a PIL.Image
        if isinstance(image_field, Image.Image):
            return ("pil", image_field)
        # If it's bytes
        if isinstance(image_field, (bytes, bytearray)):
            return ("bytes", bytes(image_field))
        # If it's a string (likely a URL or local path)
        if isinstance(image_field, str):
            if image_field.startswith("http://") or image_field.startswith("https://"):
                return ("url", image_field)
            else:
                return ("local_path", image_field)
        # If it's a dict-like
        if isinstance(image_field, dict):
            # common keys: 'path', 'file', 'url', 'image', 'image_url'
            for k in ("url", "image_url", "src", "path", "file", "filename"):
                if k in image_field and image_field[k]:
                    val = image_field[k]
                    if isinstance(val, str):
                        if val.startswith("http://") or val.startswith("https://"):
                            return ("url", val)
                        else:
                            return ("local_path", val)
            # maybe nested bytes or PIL
            if "bytes" in image_field and image_field["bytes"]:
                return ("bytes", image_field["bytes"])
            # unknown dict case: try to stringify
            return ("unknown_dict", image_field)

    # try other common keys
    for k in ("image_url", "img_url", "img", "image_path", "image_filename", "url"):
        if k in item and item[k]:
            val = item[k]
            if isinstance(val, str):
                if val.startswith("http://") or val.startswith("https://"):
                    return ("url", val)
                else:
                    return ("local_path", val)
            if isinstance(val, Image.Image):
                return ("pil", val)
            if isinstance(val, (bytes, bytearray)):
                return ("bytes", bytes(val))

    # search for any field name containing 'image' or 'img'
    for key, val in item.items():
        if key and ("image" in key.lower() or "img" in key.lower()):
            if isinstance(val, str):
                if val.startswith("http://") or val.startswith("https://"):
                    return ("url", val)
                else:
                    return ("local_path", val)
            if isinstance(val, Image.Image):
                return ("pil", val)
            if isinstance(val, (bytes, bytearray)):
                return ("bytes", bytes(val))
            if isinstance(val, dict):
                # try same logic
                for k2 in ("url", "path", "image_url"):
                    if k2 in val and isinstance(val[k2], str):
                        if val[k2].startswith("http"):
                            return ("url", val[k2])
                        else:
                            return ("local_path", val[k2])
                if "bytes" in val:
                    return ("bytes", val["bytes"])
    # no image found
    return ("none", None)

def get_image_bytes_from_source(src_type, src_value):
    """
    Return raw PNG bytes (or bytes convertible to PNG) for writing into zip.
    Ensures image bytes are PNG-formatted.
    """
    # If PIL Image -> save to PNG bytes
    if src_type == "pil":
        bio = io.BytesIO()
        src_value.save(bio, format="PNG")
        return bio.getvalue()
    if src_type == "bytes":
        # Try to load via PIL to ensure PNG format (and convert if necessary)
        try:
            bio_in = io.BytesIO(src_value)
            img = Image.open(bio_in)
            out = io.BytesIO()
            img.save(out, format="PNG")
            return out.getvalue()
        except Exception:
            # bytes not image -> return as-is
            return bytes(src_value)
    if src_type == "local_path":
        # read file from disk
        path = src_value
        if not os.path.isabs(path):
            # sometimes dataset returns relative cache path — try as-is
            path = os.path.expanduser(path)
        try:
            with open(path, "rb") as f:
                content = f.read()
            # normalize via PIL
            try:
                img = Image.open(io.BytesIO(content))
                out = io.BytesIO()
                img.save(out, format="PNG")
                return out.getvalue()
            except Exception:
                return content
        except Exception as e:
            raise RuntimeError(f"Could not read local image path '{src_value}': {e}")
    if src_type == "url":
        url = src_value
        try:
            resp = requests.get(url, stream=True, timeout=REQUESTS_TIMEOUT)
            resp.raise_for_status()
            content = resp.content
            # normalize via PIL
            try:
                img = Image.open(io.BytesIO(content))
                out = io.BytesIO()
                img.save(out, format="PNG")
                return out.getvalue()
            except Exception:
                # if not a recognizable image, return raw bytes
                return content
        except Exception as e:
            raise RuntimeError(f"Failed to download image from {url}: {e}")
    if src_type == "unknown_dict":
        # try to serialize or find nested url/path
        if isinstance(src_value, dict):
            for k in ("url", "image_url", "path"):
                v = src_value.get(k)
                if isinstance(v, str):
                    if v.startswith("http"):
                        return get_image_bytes_from_source("url", v)
                    else:
                        return get_image_bytes_from_source("local_path", v)
        raise RuntimeError("Unknown dict image format and no url/path found.")
    if src_type == "none":
        raise RuntimeError("No image source available.")
    # fallback
    raise RuntimeError(f"Unhandled src_type: {src_type}")

def make_placeholder_png_bytes(size=(1,1)):
    img = Image.new("RGBA", size, (0,0,0,0))
    bio = io.BytesIO()
    img.save(bio, format="PNG")
    return bio.getvalue()

# ---------- main logic ----------
def main():
    try:
        ds = load_phyx_test_mini()
    except Exception as e:
        print("ERROR: could not load dataset:", e)
        traceback.print_exc()
        sys.exit(1)

    # convert to list (so we can iterate twice)
    try:
        examples = list(ds)
    except Exception:
        examples = [x for x in ds]

    n = len(examples)
    print(f"Loaded dataset split with {n} examples.")

    records = []
    current_id = START_ID

    # create an in-memory zip (but we will write to disk at the end)
    with zipfile.ZipFile(OUTPUT_ZIP, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
        # iterate with progress
        for idx, item in enumerate(tqdm(examples, desc="Processing examples", unit="ex")):
            try:
                # Build json record
                text = build_text(item)
                options = item.get("options") or []
                answer_val = normalize_answer(item.get("answer"))

                record = {
                    "id": current_id,
                    "image": f"{IMAGES_DIR_IN_ZIP}/image{current_id}.png",
                    "text": text,
                    "options": options,
                    "type": "MCQs with One Correct Answer",
                    "correct_answer": answer_val
                }
                records.append(record)

                # Determine image source
                src_type, src_value = find_image_source(item)

                try:
                    img_bytes = get_image_bytes_from_source(src_type, src_value)
                except Exception as e_img:
                    # log and fallback to placeholder
                    print(f"Warning: could not get image for example index {idx} (id={current_id}): {e_img}")
                    img_bytes = make_placeholder_png_bytes(PLACEHOLDER_SIZE)

                # Write image bytes into zip at desired path
                img_path_in_zip = f"{IMAGES_DIR_IN_ZIP}/image{current_id}.png"
                zf.writestr(img_path_in_zip, img_bytes)

            except Exception as e:
                print(f"ERROR processing example index {idx}: {e}")
                traceback.print_exc()
                # still add a placeholder record so indexing remains consistent
                record = {
                    "id": current_id,
                    "image": f"{IMAGES_DIR_IN_ZIP}/image{current_id}.png",
                    "text": "",
                    "options": [],
                    "type": "MCQs with One Correct Answer",
                    "correct_answer": None
                }
                records.append(record)
                zf.writestr(f"{IMAGES_DIR_IN_ZIP}/image{current_id}.png", make_placeholder_png_bytes(PLACEHOLDER_SIZE))

            current_id += 1

        # After images are written, write the JSON file into the zip
        json_bytes = json.dumps(records, ensure_ascii=False, indent=2).encode("utf-8")
        zf.writestr(OUTPUT_JSON_NAME, json_bytes)

    # done
    abs_zip = os.path.abspath(OUTPUT_ZIP)
    print(f"Finished. Created ZIP: {abs_zip}")
    print(f" - {len(records)} records written (IDs {START_ID}..{current_id-1})")
    print(f" - JSON file inside ZIP: {OUTPUT_JSON_NAME}")
    print(f" - Images inside ZIP at: {IMAGES_DIR_IN_ZIP}/image{{ID}}.png")

if __name__ == "__main__":
    main()


Package 'Pillow' not found — installing...
Installed 'Pillow'.
Attempting: load_dataset('Cloudriver/PhyX', split='test_mini') ...
Loaded dataset split with 1000 examples.


Processing examples: 100%|██████████| 1000/1000 [00:28<00:00, 35.28ex/s]

Finished. Created ZIP: /content/phyx_images_and_data.zip
 - 1000 records written (IDs 690..1689)
 - JSON file inside ZIP: phyx_test_mini_custom.json
 - Images inside ZIP at: images/image{ID}.png


In [ ]:

import json
import os
import sys
import subprocess
import traceback

def ensure_package(pkg):
    try:
        __import__(pkg)
    except Exception:
        print(f"Package '{pkg}' not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        print(f"Installed '{pkg}'.")

def try_load_dataset():
    from datasets import load_dataset
    attempts = []
    # Try multiple common approaches to load the split
    try:
        print("Attempt: load_dataset('Cloudriver/PhyX', split='test')")
        ds = load_dataset("Cloudriver/PhyX", split="test")
        return ds
    except Exception as e:
        attempts.append(("split=test", e))
    try:
        print("Attempt: load_dataset('Cloudriver/PhyX', 'default', split='test')")
        ds = load_dataset("Cloudriver/PhyX", "default", split="test")
        return ds
    except Exception as e:
        attempts.append(("config default + split", e))
    try:
        print("Attempt: load_dataset('Cloudriver/PhyX') and select ['test']")
        ds_all = load_dataset("Cloudriver/PhyX")
        if "test" in ds_all:
            return ds_all["test"]
        else:
            raise RuntimeError(f"'test' not found in dataset splits: {list(ds_all.keys())}")
    except Exception as e:
        attempts.append(("load full dataset", e))

    # If all attempts failed, raise a combined error with traces
    msg_lines = ["Failed to load dataset. Attempts:"]
    for name, err in attempts:
        msg_lines.append(f"--- {name} error: {repr(err)}")
    raise RuntimeError("\n".join(msg_lines))

def build_json_records(dataset, start_id=1690):
    out = []
    cur = start_id
    # convert to list (some datasets are iterable and len() won't work)
    try:
        data_list = list(dataset)
    except Exception:
        # fallback: iterate
        data_list = []
        for i, row in enumerate(dataset):
            data_list.append(row)
    for item in data_list:
        # item may be a dict-like; use .get if available
        if hasattr(item, "get"):
            q = item.get("question") or ""
            qd = item.get("question_description") or ""
            options = item.get("options") or []
            answer = item.get("answer", None)
        else:
            # attempt attribute access
            q = getattr(item, "question", "") or ""
            qd = getattr(item, "question_description", "") or ""
            options = getattr(item, "options", []) or []
            answer = getattr(item, "answer", None)

        if q and qd:
            text = q.strip() + "\n\n" + qd.strip()
        else:
            text = (q + qd).strip()

        # normalize answer if it's list/tuple
        if isinstance(answer, (list, tuple)):
            answer_val = answer[0] if answer else None
        else:
            answer_val = answer

        record = {
            "id": cur,
            "image": f"images/image{cur}.png",
            "text": text,
            "options": options,
            "type": "MCQs with One Correct Answer",
            "correct_answer": answer_val,
        }
        out.append(record)
        cur += 1
    return out

def main(output_path="phyx_test_custom.json", start_id=1690):
    try:
        ensure_package("datasets")
        ensure_package("huggingface-hub")
        # import after ensure
        from datasets import __version__ as dver
        print(f"datasets version: {dver}")
    except Exception as e:
        print("Failed to ensure datasets/huggingface-hub packages:", e)
        traceback.print_exc()
        return

    try:
        print("Loading dataset Cloudriver/PhyX (test)...")
        ds = try_load_dataset()
    except Exception as e:
        print("ERROR while loading dataset:")
        traceback.print_exc()
        # write an empty file and return to indicate failure but produce output
        abs_path = os.path.abspath(output_path)
        print(f"Writing empty JSON [] to {abs_path} to avoid silent failure.")
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump([], f, ensure_ascii=False, indent=2)
        return

    # convert to list and print some metadata
    try:
        ds_list = list(ds)
        n = len(ds_list)
        print(f"Successfully loaded dataset split. Number of examples: {n}")
    except Exception as e:
        print("Could not convert dataset to list; attempting to iterate.")
        traceback.print_exc()
        ds_list = []
        for i, r in enumerate(ds):
            ds_list.append(r)
        n = len(ds_list)
        print(f"Number of examples after iteration: {n}")

    print("Building JSON records...")
    records = build_json_records(ds_list, start_id=start_id)

    # write file
    abs_path = os.path.abspath(output_path)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)

    print(f"Wrote {len(records)} records to {abs_path}")
    if records:
        print("Preview of first record:")
        print(json.dumps(records[0], ensure_ascii=False, indent=2))
    else:
        print("No records produced. Check dataset loading above.")

if __name__ == "__main__":
    main()


Package 'huggingface-hub' not found. Installing...
Installed 'huggingface-hub'.
datasets version: 4.0.0
Loading dataset Cloudriver/PhyX (test)...
Attempt: load_dataset('Cloudriver/PhyX', split='test')
Successfully loaded dataset split. Number of examples: 3000
Building JSON records...
Wrote 3000 records to /content/phyx_test_custom.json
Preview of first record:
{
  "id": 1690,
  "image": "images/image1690.png",
  "text": "How large should these pulls be?\n\nA patient with a dislocated shoulder is put into a traction apparatus as shown in figure. The pulls $\\vec{A}$ and $\\vec{B}have equal magnitudes and must combine to produce an outward traction force of 12.8 N on the patient’s arm.",
  "options": [
    "A: 7.55N",
    "B: 5.55N",
    "C: 7.65N",
    "D: 6.65N"
  ],
  "type": "MCQs with One Correct Answer",
  "correct_answer": "A"
}


In [ ]:


import io
import json
import os
import sys
import subprocess
import zipfile
import traceback

# ---------- helper to ensure packages ----------
def ensure_package(pkg_name, import_name=None):
    import_name = import_name or pkg_name
    try:
        __import__(import_name)
    except Exception:
        print(f"Package '{pkg_name}' not found — installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        print(f"Installed '{pkg_name}'.")

# ensure required packages
for pkg in ("datasets", "Pillow", "requests", "tqdm"):
    ensure_package(pkg)

from datasets import load_dataset
from PIL import Image
import requests
from tqdm import tqdm

# ---------- configuration ----------
START_ID = 1690
OUTPUT_ZIP = "phyx_images_and_data.zip"
OUTPUT_JSON_NAME = "phyx_test_custom.json"
IMAGES_DIR_IN_ZIP = "images"  # images stored under images/image{ID}.png
PLACEHOLDER_SIZE = (1, 1)  # 1x1 PNG placeholder for missing images
REQUESTS_TIMEOUT = 15  # seconds

# ---------- utility functions ----------
def load_phyx_test():
    """Try multiple ways to load the test split."""
    attempts = []
    try:
        print("Attempting: load_dataset('Cloudriver/PhyX', split='test') ...")
        ds = load_dataset("Cloudriver/PhyX", split="test")
        return ds
    except Exception as e:
        attempts.append(("split=test", e))
    try:
        print("Attempting: load_dataset('Cloudriver/PhyX', 'default', split='test') ...")
        ds = load_dataset("Cloudriver/PhyX", "default", split="test")
        return ds
    except Exception as e:
        attempts.append(("config default + split", e))
    try:
        print("Attempting: load_dataset('Cloudriver/PhyX') then select ['test'] ...")
        ds_all = load_dataset("Cloudriver/PhyX")
        if "test" in ds_all:
            return ds_all["test"]
        else:
            raise RuntimeError(f"'test' not found in splits: {list(ds_all.keys())}")
    except Exception as e:
        attempts.append(("load full dataset", e))
    # raise combined error
    msg = "Failed to load dataset. Attempts:\n"
    for name, err in attempts:
        msg += f" - {name}: {repr(err)}\n"
    raise RuntimeError(msg)

def build_text(item):
    q = item.get("question") or ""
    qd = item.get("question_description") or ""
    if q and qd:
        return q.strip() + "\n\n" + qd.strip()
    return (q + qd).strip()

def normalize_answer(ans):
    if isinstance(ans, (list, tuple)):
        return ans[0] if ans else None
    return ans

def find_image_source(item):
    """
    Try to find an image source inside the dataset example.
    Returns a tuple (type, value) where type in {'url','local_path','pil','bytes','none'}.
    """
    # direct 'image' field
    image_field = item.get("image")
    if image_field is not None:
        # If it's a PIL.Image
        if isinstance(image_field, Image.Image):
            return ("pil", image_field)
        # If it's bytes
        if isinstance(image_field, (bytes, bytearray)):
            return ("bytes", bytes(image_field))
        # If it's a string (likely a URL or local path)
        if isinstance(image_field, str):
            if image_field.startswith("http://") or image_field.startswith("https://"):
                return ("url", image_field)
            else:
                return ("local_path", image_field)
        # If it's a dict-like
        if isinstance(image_field, dict):
            # common keys: 'path', 'file', 'url', 'image', 'image_url'
            for k in ("url", "image_url", "src", "path", "file", "filename"):
                if k in image_field and image_field[k]:
                    val = image_field[k]
                    if isinstance(val, str):
                        if val.startswith("http://") or val.startswith("https://"):
                            return ("url", val)
                        else:
                            return ("local_path", val)
            # maybe nested bytes or PIL
            if "bytes" in image_field and image_field["bytes"]:
                return ("bytes", image_field["bytes"])
            # unknown dict case: try to stringify
            return ("unknown_dict", image_field)

    # try other common keys
    for k in ("image_url", "img_url", "img", "image_path", "image_filename", "url"):
        if k in item and item[k]:
            val = item[k]
            if isinstance(val, str):
                if val.startswith("http://") or val.startswith("https://"):
                    return ("url", val)
                else:
                    return ("local_path", val)
            if isinstance(val, Image.Image):
                return ("pil", val)
            if isinstance(val, (bytes, bytearray)):
                return ("bytes", bytes(val))

    # search for any field name containing 'image' or 'img'
    for key, val in item.items():
        if key and ("image" in key.lower() or "img" in key.lower()):
            if isinstance(val, str):
                if val.startswith("http://") or val.startswith("https://"):
                    return ("url", val)
                else:
                    return ("local_path", val)
            if isinstance(val, Image.Image):
                return ("pil", val)
            if isinstance(val, (bytes, bytearray)):
                return ("bytes", bytes(val))
            if isinstance(val, dict):
                # try same logic
                for k2 in ("url", "path", "image_url"):
                    if k2 in val and isinstance(val[k2], str):
                        if val[k2].startswith("http"):
                            return ("url", val[k2])
                        else:
                            return ("local_path", val[k2])
                if "bytes" in val:
                    return ("bytes", val["bytes"])
    # no image found
    return ("none", None)

def get_image_bytes_from_source(src_type, src_value):
    """
    Return raw PNG bytes (or bytes convertible to PNG) for writing into zip.
    Ensures image bytes are PNG-formatted.
    """
    # If PIL Image -> save to PNG bytes
    if src_type == "pil":
        bio = io.BytesIO()
        src_value.save(bio, format="PNG")
        return bio.getvalue()
    if src_type == "bytes":
        # Try to load via PIL to ensure PNG format (and convert if necessary)
        try:
            bio_in = io.BytesIO(src_value)
            img = Image.open(bio_in)
            out = io.BytesIO()
            img.save(out, format="PNG")
            return out.getvalue()
        except Exception:
            # bytes not image -> return as-is
            return bytes(src_value)
    if src_type == "local_path":
        # read file from disk
        path = src_value
        if not os.path.isabs(path):
            # sometimes dataset returns relative cache path — try as-is
            path = os.path.expanduser(path)
        try:
            with open(path, "rb") as f:
                content = f.read()
            # normalize via PIL
            try:
                img = Image.open(io.BytesIO(content))
                out = io.BytesIO()
                img.save(out, format="PNG")
                return out.getvalue()
            except Exception:
                return content
        except Exception as e:
            raise RuntimeError(f"Could not read local image path '{src_value}': {e}")
    if src_type == "url":
        url = src_value
        try:
            resp = requests.get(url, stream=True, timeout=REQUESTS_TIMEOUT)
            resp.raise_for_status()
            content = resp.content
            # normalize via PIL
            try:
                img = Image.open(io.BytesIO(content))
                out = io.BytesIO()
                img.save(out, format="PNG")
                return out.getvalue()
            except Exception:
                # if not a recognizable image, return raw bytes
                return content
        except Exception as e:
            raise RuntimeError(f"Failed to download image from {url}: {e}")
    if src_type == "unknown_dict":
        # try to serialize or find nested url/path
        if isinstance(src_value, dict):
            for k in ("url", "image_url", "path"):
                v = src_value.get(k)
                if isinstance(v, str):
                    if v.startswith("http"):
                        return get_image_bytes_from_source("url", v)
                    else:
                        return get_image_bytes_from_source("local_path", v)
        raise RuntimeError("Unknown dict image format and no url/path found.")
    if src_type == "none":
        raise RuntimeError("No image source available.")
    # fallback
    raise RuntimeError(f"Unhandled src_type: {src_type}")

def make_placeholder_png_bytes(size=(1,1)):
    img = Image.new("RGBA", size, (0,0,0,0))
    bio = io.BytesIO()
    img.save(bio, format="PNG")
    return bio.getvalue()

# ---------- main logic ----------
def main():
    try:
        ds = load_phyx_test()
    except Exception as e:
        print("ERROR: could not load dataset:", e)
        traceback.print_exc()
        sys.exit(1)

    # convert to list (so we can iterate twice)
    try:
        examples = list(ds)
    except Exception:
        examples = [x for x in ds]

    n = len(examples)
    print(f"Loaded dataset split with {n} examples.")

    records = []
    current_id = START_ID

    # create an in-memory zip (but we will write to disk at the end)
    with zipfile.ZipFile(OUTPUT_ZIP, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
        # iterate with progress
        for idx, item in enumerate(tqdm(examples, desc="Processing examples", unit="ex")):
            try:
                # Build json record
                text = build_text(item)
                options = item.get("options") or []
                answer_val = normalize_answer(item.get("answer"))

                record = {
                    "id": current_id,
                    "image": f"{IMAGES_DIR_IN_ZIP}/image{current_id}.png",
                    "text": text,
                    "options": options,
                    "type": "MCQs with One Correct Answer",
                    "correct_answer": answer_val
                }
                records.append(record)

                # Determine image source
                src_type, src_value = find_image_source(item)

                try:
                    img_bytes = get_image_bytes_from_source(src_type, src_value)
                except Exception as e_img:
                    # log and fallback to placeholder
                    print(f"Warning: could not get image for example index {idx} (id={current_id}): {e_img}")
                    img_bytes = make_placeholder_png_bytes(PLACEHOLDER_SIZE)

                # Write image bytes into zip at desired path
                img_path_in_zip = f"{IMAGES_DIR_IN_ZIP}/image{current_id}.png"
                zf.writestr(img_path_in_zip, img_bytes)

            except Exception as e:
                print(f"ERROR processing example index {idx}: {e}")
                traceback.print_exc()
                # still add a placeholder record so indexing remains consistent
                record = {
                    "id": current_id,
                    "image": f"{IMAGES_DIR_IN_ZIP}/image{current_id}.png",
                    "text": "",
                    "options": [],
                    "type": "MCQs with One Correct Answer",
                    "correct_answer": None
                }
                records.append(record)
                zf.writestr(f"{IMAGES_DIR_IN_ZIP}/image{current_id}.png", make_placeholder_png_bytes(PLACEHOLDER_SIZE))

            current_id += 1

        # After images are written, write the JSON file into the zip
        json_bytes = json.dumps(records, ensure_ascii=False, indent=2).encode("utf-8")
        zf.writestr(OUTPUT_JSON_NAME, json_bytes)

    # done
    abs_zip = os.path.abspath(OUTPUT_ZIP)
    print(f"Finished. Created ZIP: {abs_zip}")
    print(f" - {len(records)} records written (IDs {START_ID}..{current_id-1})")
    print(f" - JSON file inside ZIP: {OUTPUT_JSON_NAME}")
    print(f" - Images inside ZIP at: {IMAGES_DIR_IN_ZIP}/image{{ID}}.png")

if __name__ == "__main__":
    main()


Package 'Pillow' not found — installing...
Installed 'Pillow'.
Attempting: load_dataset('Cloudriver/PhyX', split='test') ...
Loaded dataset split with 3000 examples.


Processing examples: 100%|██████████| 3000/3000 [01:21<00:00, 36.88ex/s]


Finished. Created ZIP: /content/phyx_images_and_data.zip
 - 3000 records written (IDs 1690..4689)
 - JSON file inside ZIP: phyx_test_custom.json
 - Images inside ZIP at: images/image{ID}.png
